<a href="https://colab.research.google.com/github/nylee16/opensource_project/blob/main/2415485/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import os
import json
import pandas as pd
from glob import glob
from tqdm import tqdm

# 경로 설정
wav_dir = "/content/drive/MyDrive/wav_16k"
json_dir = "/content/drive/MyDrive/merged_json"
txt_dir = "/content/drive/MyDrive/merged_txt"
output_csv = "/content/drive/MyDrive/labels.csv"

data = []
json_files = glob(os.path.join(json_dir, "*.json"))

for json_path in tqdm(json_files, desc="처리 중"):
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            content = json.load(f)

        wav_name = content["original"]
        base_name = os.path.splitext(wav_name)[0]

        wav_path = os.path.join(wav_dir, wav_name)
        txt_path = os.path.join(txt_dir, base_name + ".txt")

        if not os.path.exists(wav_path):
            print(f"⚠️ Wav 없음: {wav_path}")
            continue
        if not os.path.exists(txt_path):
            print(f"⚠️ Txt 없음: {txt_path}")
            continue

        try:
            with open(txt_path, "r", encoding="utf-8") as t:
                transcript = t.read().strip()
        except UnicodeDecodeError:
            # 인코딩 문제 있을 경우 재시도
            try:
                with open(txt_path, "r", encoding="utf-16") as t:
                    transcript = t.read().strip()
            except Exception as e:
                print(f"❌ 텍스트 파일 읽기 실패: {txt_path} → {e}")
                continue

        data.append({"file_path": wav_path, "transcript": transcript})

    except Exception as e:
        print(f"❌ 오류 발생 in {json_path}: {e}")

# 저장
df = pd.DataFrame(data)
df.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"✅ CSV 저장 완료: {output_csv} | 총 {len(df)}개 샘플")


처리 중: 100%|██████████| 1500/1500 [11:41<00:00,  2.14it/s]

✅ CSV 저장 완료: /content/drive/MyDrive/labels.csv | 총 1500개 샘플
